требуется реализовать полноценный класс батч-нормализации без использования стандартной функции, принимающий на вход двумерный тензор. Обратите внимание: расчёт дисперсии ведётся по смещенной выборке, а расчет скользящего среднего по несмещенной.

In [77]:
import torch
import torch.nn as nn

input_size = 3
batch_size = 5
eps = 1e-1


class CustomBatchNorm1d:
    def __init__(self, weight, bias, eps, momentum):
        self.predict=False
        self.G = weight
        self.B = bias
        self.eps = eps
        self.momentum = momentum
        self.M = torch.Tensor(input_size).zero_()
        self.D = torch.Tensor(input_size).fill_(1)

        
    def __call__(self, input_tensor):
        if self.predict==False:
            normed_tensor = self.G*((input_tensor - input_tensor.mean(dim=0)) / torch.sqrt(input_tensor.var(dim=0, unbiased=False) + self.eps)) + self.B
            self.M = (1 - self.momentum) * self.M + input_tensor.mean(dim=0) * self.momentum
            self.D = ((1 - self.momentum) * input_tensor.var(dim=0, unbiased=False) * batch_size) / (batch_size - 1) + self.momentum * self.D
        else:
            normed_tensor = self.G*((input_tensor - self.M) / torch.sqrt(self.D + self.eps)) + self.B
        return normed_tensor

    def eval(self):
        self.predict = True

batch_norm = nn.BatchNorm1d(input_size, eps=eps)
batch_norm.bias.data = torch.randn(input_size, dtype=torch.float)
batch_norm.weight.data = torch.randn(input_size, dtype=torch.float)
batch_norm.momentum = 0.5

custom_batch_norm1d = CustomBatchNorm1d(batch_norm.weight.data, batch_norm.bias.data, eps, batch_norm.momentum)



In [78]:
# Проверка происходит автоматически вызовом следующего кода

all_correct = True

for i in range(8):
    torch_input = torch.randn(batch_size, input_size, dtype=torch.float)
    norm_output = batch_norm(torch_input)
    custom_output = custom_batch_norm1d(torch_input)
    all_correct &= torch.allclose(norm_output, custom_output) \
        and norm_output.shape == custom_output.shape

batch_norm.eval()
custom_batch_norm1d.eval()

for i in range(8):
    torch_input = torch.randn(batch_size, input_size, dtype=torch.float)
    norm_output = batch_norm(torch_input)
    custom_output = custom_batch_norm1d(torch_input)
    all_correct &= torch.allclose(norm_output, custom_output) \
        and norm_output.shape == custom_output.shape
print(all_correct)

True
